# Programming Assignment 2

In this assignment, you will:

1. Practice writing code to train and evaluate models using both the two-way holdout method, and an evaluation approach appropriate for models with hyperparameters that uses k-fold cross validation plus a test set.

2. Practice writing code to optimize a machine learning model. In particular, we will use gradient descent to optimize a logistic regression model.

3. **574 Only**: Perform optimization with a different algorithm (Newton-Raphson)

# Resources you can use to complete this assignment (a COMPLETE list)

**NOTE: You ARE allowed to use Google to find things that fit this list (i.e. it is often easy to google something like "plotly draw line graph" to find the right part of the plotly documentation).**

- Anything linked to in this article
- Anything linked to from the course web page
- Any materials from another online course taught at a university (**if you use this, you MUST provide a link to the exact document used**)
- Anything posted by Kenny, Navid, or Yincheng on Piazza



# Grading

For grading of code in Part 2, we will execute the submitted notebook as follows:

```shell
jupyter nbconvert --to python Assignment2-Student.ipynb
python Assignment2-Student.py
```

The PDF that comes along with this document has other details on the points awarded for each part.

As such, you will submit, one member of your group will subit as a zip file on UBLearns, a ```.zip``` file that contains 4 things:
- Your completed jupyter notebook.
- Your written report, answering all questions asked here (and copied in the assignment PDF)
- `part_1.1_results.csv`
- `part_1.4_results.csv`


# Part 1 - Predicting Review Scores on Pitchfork

For Part 1, we will be using data from [this paper](https://ojs.aaai.org/index.php/ICWSM/article/view/7355). The data is a collection of reviews from [Pitchfork](https://pitchfork.com/), a site that provides expert reviews of music album. The authors of this paper have also combined the data with a set of features from [Spotify’s API](https://developer.spotify.com/documentation/web-api/) that provide insight into the music itself, e.g. the "acousticness" of the song.  We will tackle a regression problem here, trying to predict the score of a review from several of the other columns in the dataset.

## Part 1.1 - Feature Engineering with Feature Subsets

In the first subsection of Part 1, we’re going to rely on our old friend linear regression. We’re going to look at how running linear regression with various subsets of our features impacts our ability to predict score.

In Part 1.1, your task is to write code below that trains a separate linear regression model for a number of different feature subsets.  Specifically:

- The list `feature_sets` below is a list of lists; each sublist is a different subset of features to build a model with. 
- All models should be trained on the dataset `part1_train.csv`. 
- For each of these trained models, you should evaluate the model’s predictions on the training dataset, as well as the provided test set, called `part1_test.csv`. The evaluation metric we will use is **root mean squared error**.  

Write out the result to a file called `part_1.1_results.csv` and submit this along with your assignment. The file should have the following columns:
- `feature_set` - a column describing the features of the model used. For feature sets with multiple features, combine them using an underscore (you can do this with the code `"_".join(feature_set)`)
- `training_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the training data
- `test_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the test data

In addition, please answer the following questions:
- **1.1.1** Which model had the best RMSE on the *training data*? **['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre', 'danceability', 
  'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
  'liveness', 'valence', 'tempo']**
- **1.1.2** Which model had the best RMSE on the *test data*? **same as 1.1.1** 
- **1.1.3** Which feature do you believe was the most important one? Why? *(Note: There is more than one perfectly acceptable way to answer this question)* **release year, because it has the lowest rmse both in training and test among single features**
- **1.1.4** What can we say about the utility of the Spotify features based on these results? **the more features, the more predictable. Spotify features could help predict the scores correctly**

In [109]:
feature_sets = [['artist'],
 ['reviewauthor'],
 ['releaseyear'],
 ['recordlabel'],
 ['genre'],
 ['danceability'],
 ['energy'],
 ['key'],
 ['loudness'],
 ['speechiness'],
 ['acousticness'],
 ['instrumentalness'],
 ['liveness'],
 ['valence'],
 ['tempo'],
 ['danceability','energy','key','loudness','speechiness','acousticness',
  'instrumentalness','liveness','valence','tempo'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre', 'danceability', 
  'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
  'liveness', 'valence', 'tempo']]


In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

# Convenience things for you, note that releaseyear is continuous but is not a Spotify API variable
CONTINUOUS_FEATURES = ['releaseyear', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']
CATEGORICAL_FEATURES = ['artist', 'reviewauthor', 'recordlabel', 'genre']

# Read in the data
training_data = pd.read_csv("part1_train.csv")
test_data = pd.read_csv("part1_test.csv")

from sklearn.preprocessing import StandardScaler
training_data[CONTINUOUS_FEATURES] = StandardScaler().fit_transform(training_data[CONTINUOUS_FEATURES].values)
test_data[CONTINUOUS_FEATURES] = StandardScaler().fit_transform(test_data[CONTINUOUS_FEATURES].values)

from sklearn.preprocessing import LabelEncoder
for col in CATEGORICAL_FEATURES:
  le = LabelEncoder()
  
  le.fit(training_data[col])
  training_data[col] = le.transform(training_data[col])

  le.fit(test_data[col])
  test_data[col] = le.transform(test_data[col])

  
result_data = []
for feature_set in feature_sets:
    # Write your code for Part 1.1 here!

    training_data_feature = '_'.join(feature_set) 

    X_train = training_data[feature_set]
    y_train = training_data['score']
   
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    train_pred = lr.predict(X_train)
    
    X_test = test_data[feature_set]
    y_test = test_data['score']

    y_pred = lr.predict(X_test)

    training_rmse = np.sqrt(((train_pred - y_train) **2).mean())
    test_rmse = np.sqrt(((y_pred - y_test) ** 2).mean())

    result_data.append([training_data_feature, training_rmse, test_rmse])
    
df = pd.DataFrame(result_data, columns = ['feature_set','training_rmse','test_rmse'])
df.to_csv('part_1.1_results.csv',index=False)


In [111]:
df

,feature_set,training_rmse,test_rmse
0,artist,1.243393,1.243591
1,reviewauthor,1.243345,1.243498
2,releaseyear,1.235994,1.232972
3,recordlabel,1.242969,1.244257
4,genre,1.242326,1.242353
5,danceability,1.240362,1.241122
6,energy,1.240900,1.239846
7,key,1.243497,1.243804
8,loudness,1.236968,1.237561
9,speechiness,1.243530,1.243566


## Part 1.2 - Feature Engineering with the LASSO

In Part 1.2, your task is to write code below that trains an L1-regularized linear regression model, with an expanded feature set.  Specifically:

1. Begin with the final feature set listed in `feature_sets` (i.e. your feature set, to begin this section, is `feature_sets[-1]`.
2. One-hot encode your categorical variables, setting `drop=if_binary` and `sparse=False` in the function arguments. 
3. Scale all of your continuous features using the `StandardScaler`.
4. Train an L1-regularized linear regression model using these features on the dataset `part1_train.csv`. You should use the [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) class in `sklearn`, it will do the cross-validation necessary to select the appropriate value for the regularizer for you!  Use 10-fold cross-validation to perform model selection (set the `LassoCV` parmaeter `cv` to 10), and set the `random_state` to 1. Do not change any of the other parameters to `LassoCV` (i.e. leave them at their defaults).
5. Identify the best `alpha` value (the regularizer term, according to `sklearn`. In class, we refer to this as $\lambda$!) in terms of average mean squared error according to the cross-validation.
6. Finally, train a [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) model on the entire training dataset (`part1_train.csv`). You will use this to report the root mean squared error on the test set (Question 1.2.4 below), and use it in Part 1.3 below as well.

**Hint: The proceedure outlined above is very similar to ones we have discussed in class and shown how to do using `Pipeline`s.** 


In [127]:
# Write your code for Part 1.2 here
from sklearn.linear_model import LassoCV, Lasso
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

# Do the CV to find alpha 
#1
features = feature_sets[-1]

#2
onehot = OneHotEncoder(drop='if_binary', sparse=False).fit(training_data[CATEGORICAL_FEATURES])

#3
training_data[CONTINUOUS_FEATURES]= StandardScaler().fit_transform(training_data[CONTINUOUS_FEATURES].values)

X=training_data[features]
y=training_data['score']

#4

lassoModel = LassoCV(cv=10, random_state = 1).fit(X, y)
train_rmse = np.sqrt(mean_squared_error(y, lassoModel.predict(X)))
alpha = lassoModel.alpha_

# Retrain the model
lasso = Lasso(alpha = alpha, random_state = 1)
lasso.fit(X,y)

test_data[CONTINUOUS_FEATURES]= StandardScaler().fit_transform(test_data[CONTINUOUS_FEATURES].values)
onehot = OneHotEncoder(drop='if_binary', sparse=False).fit(test_data[CATEGORICAL_FEATURES])

X_test = test_data[features]
y_test = test_data['score']
y_pred = lasso.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('lasso average RMSE:',train_rmse)
print('lasso test_data rmse:',rmse)

lasso average RMSE: 1.22447529655717
lasso test_data rmse: 1.2242650520928589


In [116]:
alpha

0.004391903636588995

Now, answer the following questions:
- **1.2.1** - How many total features are introduced by Step 2 above? Provide both the number and an explanation of how you got to this number. **16**
- **1.2.2** - What was the best `alpha` value according to your cross-validation results? **0.004391903636588995**
- **1.2.3** - What was the **average RMSE** of the model with this `alpha` value on the k-fold cross validation on the *training* data? **1.22447529655717**
- **1.2.4** - What was the **RMSE** of the model with this `alpha` value on the k-fold cross validation on the *test* data? **1.2242650520928589**


# Part 1.3 - Interpreting Model Coefficents

In this section we will interpret the coefficients from the final model you trained on all of the training data.

- **1.3.1** - How many non-zero coefficients are in this final model? **13**
- **1.3.2** - What percentage of the coefficients are non-zero in this final model? **86.66666666666667**
- **1.3.3** - Who were the three most critical review authors, as estimated by the model? How do you know? **Allison Hussey
Evan Minsker
Alphonse Pierre**
- **1.3.4** - Who were the three artists that reviewers tended to like the most?  How do you know? **Brian Eno
OTHER**
- **1.3.5** - What genre did Pitchfork reviewers tend to like the most? Which genre did they like the least? **Rap, the most. Jazz, the least**

In [126]:
# Code for 1.3 here
#1.3.1
num_nonzero = 0
for i in lasso.coef_:
  if i!=0:
    num_nonzero+=1

print(num_nonzero)

#1.3.2
print((num_nonzero/len(lasso.coef_))*100, "%")

13
86.66666666666667 %


In [83]:
ori_training_data = pd.read_csv('part1_train.csv')

In [159]:
import matplotlib.pyplot as plt

#1.3.3

df3 = pd.DataFrame(ori_training_data['reviewauthor'], columns = ['reviewauthor'])
df3['reviewauthor'] = ori_training_data['reviewauthor']
df3['predict'] = lasso.predict(training_data.drop('score',axis=1))

print(df3.sort_values(by=['predict'], axis=0))
print(ori_training_data['reviewauthor'][14515])
print(ori_training_data['reviewauthor'][577])
print(ori_training_data['reviewauthor'][14940])

               reviewauthor   predict
14515        Allison Hussey  6.438343
577            Evan Minsker  6.512743
14940       Alphonse Pierre  6.519177
8656        Dean Van Nguyen  6.521953
14299         Brian Josephs  6.529436
...                     ...       ...
11460  Grayson Haver Currin  8.123096
7580     Stephen M. Deusner  8.137757
16096    Stephen M. Deusner  8.138749
7418                  OTHER  8.146354
14584           Liam Singer  8.215757

[16270 rows x 2 columns]
Allison Hussey
Evan Minsker
Alphonse Pierre


In [161]:
#1.3.4

df4 = pd.DataFrame(ori_training_data['artist'], columns = ['artist'])
df4['artist'] = ori_training_data['artist']
df4['predict'] = lasso.predict(training_data.drop('score',axis=1))

print(df4.sort_values(by=['predict'], axis=0, ascending=False))
print(ori_training_data['artist'][14584])
print(ori_training_data['artist'][7418])

          artist   predict
14584  Brian Eno  8.215757
7418       OTHER  8.146354
16096      OTHER  8.138749
7580       OTHER  8.137757
11460      OTHER  8.123096
...          ...       ...
14299      OTHER  6.529436
8656       OTHER  6.521953
14940      OTHER  6.519177
577        OTHER  6.512743
14515      OTHER  6.438343

[16270 rows x 2 columns]
Brian Eno
OTHER


In [162]:
#1.3.5

df5 = pd.DataFrame(ori_training_data['genre'], columns = ['genre'])
df5['genre'] = ori_training_data['genre']
df5['predict'] = lasso.predict(training_data.drop('score',axis=1))

mean_df5 = df5.groupby('genre')['predict'].agg(**{'mean_predict':'mean'}).reset_index()
print(mean_df5.sort_values(by=['mean_predict'], axis=0))

          genre  mean_predict
7           Rap      6.896323
9          none      6.903372
6       Pop/R&B      6.940548
8          Rock      7.010541
3        Global      7.044791
5         Metal      7.076546
0    Electronic      7.096234
2  Folk/Country      7.111136
1  Experimental      7.164946
4          Jazz      7.174001


# Part 1.4 - "Manual" Cross-Validation + Holdout for Model Selection and Evaluation

We will finally use cross validation for both algorithm and model selection, with a hold-out test set for a final evaluation. We will use **5-fold cross validation** to identify the best parameters and hyperparameters for a set of models. We will then take our final models and use a final hold-out test set (the same one as above) to estimate the generalization error of the models.

Specifically, your task is first to write code that trains and evaluates the following models, one for each of the specified hyper parameters sets:

- `Decision Tree regression` - All combinations of a `max_depth` of 5, 10, or 20, and a `criterion` of `"squared error"` or `"absolute error"`. Use the [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor).
- Ridge regression - Use the following choices of L2 penalty: $[10^{-5}, 10^{-4}, ..., 10^4, 10^5]$. In Python, you can create a list of these numbers using `np.logspace(-5, 5, 11)`. Use the [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) class from sklearn to train a Ridge Regression model. The parameters you need to pass when constructing the Ridge model are `alpha`, which lets you specify what you want the L2 penalty to be, and `random_state=0` to avoid randomness.
- kNN regression - Values of `n_neighbors` of 1, 5, 10, and 15. Use the [KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) class.

Additional notes:
1. All models should use the feature sets described in Part 1.3 (the same ...one-hot encoded... categorical variables, and the scaled continuous variables)
2. As opposed to using the `KFold` class from `sklearn` like we did in class, we have instead provided you with pre-existing data sets; you should therefore use the pre-split data in the provided CSV files ``1.2_fold0.csv ... 1.2_fold4.csv``

**What to submit**:

1. Write out the result to a file called `part_1.4_results.csv` and submit this along with your assignment. The file should have the following columns:
- `model_name` - The name of the model, one of `DTR` (Decision Tree Regression), `Ridge`, or `KNN`.
- `hyperparameter_setting` - a column describing the hyperparameters of the model. For models with multiple hyperparameters, combine them using an underscore (you can do this with the code `"_".join(hyperparameters)`).
- `mean_training_rmse` - a column that gives the mean RMSE on the k-fold training data. You should take the average of the model’s errors on the different folds, using root mean squared error again as your evaluation metric.
- `sd_training_rmse` - a column that gives the standard deviation RMSE on the k-fold training data.
- `test_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the test data


2. Answer the following questions:
- **1.4.1** Report, for each model, the hyper parameter setting that resulted in the best performance **squared_error_5,10000.0,15**
- **1.4.2** Which model performed the best overall on the cross-validation? **KNN** 
- **1.4.3** Which model performed the best overall on the final test set? **DTR**
- **1.4.4** With respect to your answer for 1.4.3, why do you think that might be? (*Note: there is more than one correct way to answer this question*)
- **1.4.5** Which model/hyperparameter setting had the highest standard deviation across the different folds of the cross validation? **Ridge/100000.0**
- **1.4.6** With respect to your answer for 1.4.6, why do you think that might be? (*Note: there is more than one correct way to answer this question*) **If hyperparameter value is too high, it has the danger of underfitting. The hyperparmeter of Ridge for 1.4.5 was too high. So it yields highest standard deviation**

In [164]:
from scipy.sparse.construct import random
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor

result_data = []

In [234]:
fold0 = pd.read_csv('1.2_fold0.csv')
fold1 = pd.read_csv('1.2_fold1.csv')
fold2 = pd.read_csv('1.2_fold2.csv')
fold3 = pd.read_csv('1.2_fold3.csv')
fold4 = pd.read_csv('1.2_fold4.csv')

In [239]:
from sklearn.preprocessing import OneHotEncoder

def onehot_encode_var(data, varname):
    encoder = OneHotEncoder(drop = "if_binary",sparse=False)
    encoder.fit(data[varname].values.reshape(-1,1))
    onehot_encoded_variable = encoder.transform(data[varname].values.reshape(-1,1))
    return onehot_encoded_variable

In [240]:
def data_encoding_scaling(fold):
  for feature in CATEGORICAL_FEATURES:
    onehot = onehot_encode_var(fold,feature)
    fold[feature] = onehot
  fold[CONTINUOUS_FEATURES]= StandardScaler().fit_transform(fold[CONTINUOUS_FEATURES].values)
  return fold

fold0 = data_encoding_scaling(fold0)
fold1 = data_encoding_scaling(fold1)
fold2 = data_encoding_scaling(fold2)
fold3 = data_encoding_scaling(fold3)
fold4 = data_encoding_scaling(fold4)

In [242]:
def fold(a,b,c,d):
  fold_train = np.concatenate((a,b),axis=0)
  fold_train = np.concatenate((fold_train,c),axis=0)
  fold_train = pd.DataFrame(np.concatenate((fold_train,d),axis=0),columns=fold0.columns)
  return fold_train 

In [243]:
#DecisionTreeRegressor 

max_depth = [5,10,20]
criterion = ['squared_error','absolute_error']

for c in criterion:
  for m in max_depth:
    DTR = DecisionTreeRegressor(criterion=c, max_depth=m)

    train_rmse=[]
    test_rmse=[]

    #fold0 = val data
    fold_train = fold(fold1,fold2,fold3,fold4)
    DTR.fit((fold_train.drop('score',axis=1)),fold_train.score)
    train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, DTR.predict(fold_train.drop('score',axis=1)))))
    test_rmse.append(np.sqrt(mean_squared_error(fold0.score, DTR.predict(fold0.drop('score',axis=1)))))

    #fold1 = val data
    fold_train = fold(fold0,fold2,fold3,fold4)
    DTR.fit((fold_train.drop('score',axis=1)),fold_train.score)
    train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, DTR.predict(fold_train.drop('score',axis=1)))))
    test_rmse.append(np.sqrt(mean_squared_error(fold1.score, DTR.predict(fold1.drop('score',axis=1)))))

    #fold2 = val data 
    fold_train = fold(fold0,fold1,fold3,fold4)
    DTR.fit((fold_train.drop('score',axis=1)),fold_train.score)
    train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, DTR.predict(fold_train.drop('score',axis=1)))))
    test_rmse.append(np.sqrt(mean_squared_error(fold2.score, DTR.predict(fold2.drop('score',axis=1)))))

    #fold3 = val data
    fold_train = fold(fold0,fold1,fold2,fold4)
    DTR.fit((fold_train.drop('score',axis=1)),fold_train.score)
    train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, DTR.predict(fold_train.drop('score',axis=1)))))
    test_rmse.append(np.sqrt(mean_squared_error(fold3.score, DTR.predict(fold3.drop('score',axis=1)))))
    
    #fold4 = val data 
    fold_train = fold(fold0,fold1,fold2,fold3)
    DTR.fit((fold_train.drop('score',axis=1)),fold_train.score)
    train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, DTR.predict(fold_train.drop('score',axis=1)))))
    test_rmse.append(np.sqrt(mean_squared_error(fold4.score, DTR.predict(fold4.drop('score',axis=1)))))
    
    mean_training_rmse = np.mean(train_rmse)
    sd_training_rmse = np.sqrt(mean_training_rmse)

    hyperparameters=[c,str(m)]

    result_data.append({
      'model_name' : 'DTR',
      'hyperparameter_setting' : '_'.join(hyperparameters),
      'mean_training_rmse': mean_training_rmse,
      'sd_training_rmse': sd_training_rmse,
      'test_rmse':np.mean(test_rmse)})

In [244]:
#Ridge 
for alpha in np.logspace(-5, 5, 11):
  ridge = Ridge(alpha=alpha,random_state=0).fit(X,y)

  train_rmse=[]
  test_rmse=[]

  #fold0 
  fold_train = fold(fold1,fold2,fold3,fold4)
  ridge.fit((fold_train.drop('score',axis=1)),fold_train.score)
  train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, ridge.predict(fold_train.drop('score',axis=1)))))
  test_rmse.append(np.sqrt(mean_squared_error(fold0.score, ridge.predict(fold0.drop('score',axis=1)))))

  #fold1
  fold_train = fold(fold0,fold2,fold3,fold4)
  ridge.fit((fold_train.drop('score',axis=1)),fold_train.score)
  train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, ridge.predict(fold_train.drop('score',axis=1)))))
  test_rmse.append(np.sqrt(mean_squared_error(fold1.score, ridge.predict(fold1.drop('score',axis=1)))))

  #fold2
  fold_train = fold(fold0,fold1,fold3,fold4)
  ridge.fit((fold_train.drop('score',axis=1)),fold_train.score)
  train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, ridge.predict(fold_train.drop('score',axis=1)))))
  test_rmse.append(np.sqrt(mean_squared_error(fold2.score, ridge.predict(fold2.drop('score',axis=1)))))

  #fold3
  fold_train = fold(fold0,fold1,fold2,fold4)
  ridge.fit((fold_train.drop('score',axis=1)),fold_train.score)
  train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, ridge.predict(fold_train.drop('score',axis=1)))))
  test_rmse.append(np.sqrt(mean_squared_error(fold3.score, ridge.predict(fold3.drop('score',axis=1)))))

  #fold4
  fold_train = fold(fold0,fold1,fold2,fold3)
  ridge.fit((fold_train.drop('score',axis=1)),fold_train.score)
  train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, ridge.predict(fold_train.drop('score',axis=1)))))
  test_rmse.append(np.sqrt(mean_squared_error(fold4.score, ridge.predict(fold4.drop('score',axis=1)))))

  mean_training_rmse = np.mean(train_rmse)
  sd_training_rmse = np.sqrt(mean_training_rmse)

  result_data.append({
      'model_name' : 'Ridge',
      'hyperparameter_setting' : alpha,
      'mean_training_rmse': mean_training_rmse,
      'sd_training_rmse': sd_training_rmse,
      'test_rmse':np.mean(test_rmse)})

  

In [245]:
#KNN
n_neighbors = [1,5,15]
for n in n_neighbors:
  KNN = KNeighborsRegressor(n_neighbors=n)

  train_rmse=[]
  test_rmse=[]

  #fold0 
  fold_train = fold(fold1,fold2,fold3,fold4)
  KNN.fit((fold_train.drop('score',axis=1)),fold_train.score)
  train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, KNN.predict(fold_train.drop('score',axis=1)))))
  test_rmse.append(np.sqrt(mean_squared_error(fold0.score, KNN.predict(fold0.drop('score',axis=1)))))

  #fold1
  fold_train = fold(fold0,fold2,fold3,fold4)
  KNN.fit((fold_train.drop('score',axis=1)),fold_train.score)
  train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, KNN.predict(fold_train.drop('score',axis=1)))))
  test_rmse.append(np.sqrt(mean_squared_error(fold1.score, KNN.predict(fold1.drop('score',axis=1)))))

  #fold2
  fold_train = fold(fold0,fold1,fold3,fold4)
  KNN.fit((fold_train.drop('score',axis=1)),fold_train.score)
  train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, KNN.predict(fold_train.drop('score',axis=1)))))
  test_rmse.append(np.sqrt(mean_squared_error(fold2.score, KNN.predict(fold2.drop('score',axis=1)))))

  #fold3
  fold_train = fold(fold0,fold1,fold2,fold4)
  KNN.fit((fold_train.drop('score',axis=1)),fold_train.score)
  train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, KNN.predict(fold_train.drop('score',axis=1)))))
  test_rmse.append(np.sqrt(mean_squared_error(fold3.score, KNN.predict(fold3.drop('score',axis=1)))))

  #fold4
  fold_train = fold(fold0,fold1,fold2,fold3)
  KNN.fit((fold_train.drop('score',axis=1)),fold_train.score)
  train_rmse.append(np.sqrt(mean_squared_error(fold_train.score, KNN.predict(fold_train.drop('score',axis=1)))))
  test_rmse.append(np.sqrt(mean_squared_error(fold4.score, KNN.predict(fold4.drop('score',axis=1)))))

  mean_training_rmse = np.mean(train_rmse)
  sd_training_rmse = np.sqrt(mean_training_rmse)

  result_data.append({
      'model_name' : 'KNN',
      'hyperparameter_setting' : n,
      'mean_training_rmse': mean_training_rmse,
      'sd_training_rmse': sd_training_rmse,
      'test_rmse':np.mean(test_rmse)})


In [246]:
df = pd.DataFrame(result_data, columns = ['model_name','hyperparameter_setting','mean_training_rmse','sd_training_rmse','test_rmse'])
df.to_csv('part_1.4_results.csv',index=False)

In [249]:
df

,model_name,hyperparameter_setting,mean_training_rmse,sd_training_rmse,test_rmse
0,DTR,squared_error_5,1.188279,1.090082,1.211970
1,DTR,squared_error_10,1.105744,1.051544,1.277543
2,DTR,squared_error_20,0.617524,0.785827,1.608751
3,DTR,absolute_error_5,1.212710,1.101231,1.233800
4,DTR,absolute_error_10,1.155118,1.074764,1.287838
5,DTR,absolute_error_20,0.819255,0.905127,1.530941
6,Ridge,0.00001,1.225118,1.106850,1.226633
7,Ridge,0.0001,1.225118,1.106850,1.226633
8,Ridge,0.001,1.225118,1.106850,1.226633
9,Ridge,0.01,1.225118,1.106850,1.226633


# Part 2

In class, we have shifted from regression to classification. Here, we're going to get a little practice in optimizing one of the classification models we saw in class - logistic regression. As a reminder...

The loss function of logistic regression (also known as the logistic-loss or log-loss) is given by:
\begin{equation}
  J({\bf w}) = \frac{1}{n}\sum_{i=1}^n \log{(1 + \exp{(-y_i{\bf w}^\top{\bf x}_i}))}
  \label{eqn:logloss}
\end{equation}

The gradient for this loss function, as derived in class, is:
\begin{equation}
  \nabla J({\bf w}) = -\frac{1}{n}\sum_{i=1}^n \frac{y_i}{1 + \exp{(y_i{\bf w}^\top{\bf x}_i)}}{\bf x}_i
  \label{eqn:loglossgradient}
\end{equation}


The Hessian for the loss function is given by:
\begin{equation}
  {\bf H}({\bf w}) = \frac{1}{n} \sum_{i=1}^n \frac{\exp{(y_i{\bf w}^\top{\bf x}_i)}}{(1 + \exp{(y_i{\bf w}^\top{\bf x}_i)})^2}{\bf x}_i{\bf x}_i^\top
  \label{eqn:loglosshessian}
\end{equation}

## Part 2.1 - Logistic Regression with Gradient Descent

In Part 2.1 we will implement logistic regression with gradient descent. You need to finish implementing 3 functions:

1. `logistic_objective` - compute the logistic loss for the given data set (see equation above)
2. `logistic_gradient` - compute the gradient vector of logistic loss for the given data set (see equation above)
3. `run_gradient_descent` - run the gradient descent algorithm, given these two functions.

We have provided you with some simulation data to evaluate your method with. Part 2.1 will, however, largely be graded by evaluating your code on a slightly different dataset to ensure robustness. 

In addition, please submit answers to the following questions on your written report:

- **2.1.1** - How did you go about selecting a good step size, i.e. one that was not too big or too small? (*Note: There is more than one correct answer to this*)
- **2.1.2** - What is the condition under which we assume that the gradient descent algorithm has converged in the code here?
- **2.1.3** - What is a different convergence metric we could have used? (*Note: There is more than one correct answer to this*)

In [251]:
import math
def logistic_objective(w, X, y):

    # compute log-loss error (scalar) with respect
    # to w (vector) for the given data X and y                               
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = scalar
    print(X)
    print(y)
    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    n = len(y)
    sigma = 0
    for i in range(n):
      sigma += math.log10((1+math.exp(y[i]*(np.transpose(w)*X[i]))))
    error = (1/n)*sigma
    return error

def logistic_gradient(w, X, y):

    # compute the gradient of the log-loss error (vector) with respect
    # to w (vector) for the given data X and y  
    #
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = d length gradient vector (not a d x 1 matrix)

    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    n = len(y)
    sigma_gradient=0
    for i in range(n):
      sigma_gradient += (y[i])*(1/(1+math.exp(y[i]*(np.transpose(w)*X[i]))))*X[i]
    gradient = (1/n)*sigma_gradient
    return gradient

def run_gradient_descent(X,y):
    old_w = np.array([-1]*X.shape[1])
    # change this value! This is an unreasonable step size
    step_size = 100000
    new_w =old_w - 100
    
    while ((new_w-old_w)**2).sum() > .0000000001:
        #IMPLEMENT THIS!
        pass
    return new_w



In [252]:
from scipy.stats import uniform, bernoulli
import functools
draw_binary = functools.partial(np.random.binomial,n=1)

## Simulated data to test your method
DATA_SIZE = 10000
x1 = bernoulli(.5).rvs(DATA_SIZE)
x2 = np.floor(uniform(18,60).rvs(DATA_SIZE))
true_w = [-9, 3.5, 0.2]
xb = true_w[0] + true_w[1]*x1 + true_w[2]*x2
p = 1/(1 + np.exp(-xb))
y = np.array([1 if draw_binary(p=v) else -1 for v in p])

In [255]:
from sklearn.linear_model import LogisticRegression

# notice that logistic regression as implemented in sklearn does not get the exact results either!
# so you shouldn't worry if you're a bit off
X = np.hstack([np.ones((len(xb),1)), x1[:,np.newaxis], x2[:,np.newaxis]])
model = LogisticRegression(solver='liblinear', random_state=0,fit_intercept=False)
model.fit(X,y).coef_

array([[-8.64326433,  3.33431854,  0.19197429]])

In [254]:
# this is how we will test your results
gd_result = run_gradient_descent(X,y)
# is your result relatively close to the truth?
np.abs(true_w-gd_result).sum()

KeyboardInterrupt: ignored

## <span style="color:red"> 574 Only</span> Part 2.2 - Optimization with Newton-Raphson <span style="color:red"> 574 Only</span>

In Part 2.2, you are going to, instead of using gradient descent, use the Newton-Raphson method to optimize the same logistic regression model. To do so, you will need to 1) implement the `logistic_hessian` function to compute the Hessian matrix of logistic loss for the given data set, and 2) use `scipy`'s `optimize` function to perform the optimization, rather than writing a function by hand to do so.  

For Part 2.2, you will only need to implement these functions, we will test them using our own code. You can, however, perform the same kinds of tests that we proposed above to check your work! 

In [ ]:
def logistic_hessian(w, X, y):

    # compute the Hessian of the log-loss error (matrix) with respect
    # to w (vector) for the given data X and y                               
    #
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # Hessian = d x d matrix
    
    
    if len(w.shape) == 1:
        w = w[:,np.newaxis]

    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    hessian = 0
    return hessian

In [ ]:
from scipy.optimize import minimize

def run_newton_raphson(X,y):
    args = (X,y[:,np.newaxis])
    opts = {'maxiter' : 50}    # Preferred value.    
    w_init = np.zeros(X.shape[1])
    
    # note: this is *almost* what you need, you just need to figure out what arguments are necessary here!
    soln = minimize(PUT_THE_RIGHT_ARGUMENTS_IN_HERE!
                    args=args,
                    method='Newton-CG',
                    options=opts)

    w = np.transpose(np.array(soln.x))
    w = w[:,np.newaxis]
    return w


In [ ]:
run_newton_raphson(X,y)